### Préparation de l'environnement

Ci-dessous quelques imports et précautions préalables à notre travail. Il n'est pas inutile de les parcourir.
Si nécessaire créer un bloc au démarrage pour installer toutes les librairies nécessaires en exécutant chacune leur tour les commandes suivantes:

In [ ]:
# imports
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
from statistics import mean
from time import *
from pandas.plotting import scatter_matrix


# stabilité du notebook d'une exécution à l'autre
random=np.random.default_rng(42) 

# jolies figures directement dans le notebook
%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Création des dataframes

Nous commençons par importer les CSV afin de crée les dataframes correspondant.

In [ ]:
def load_data(fileName):
    csv_path = os.path.join('../data', fileName)
    df = pd.read_csv(csv_path)
    return df

In [ ]:
employee_survey_data = load_data('employee_survey_data.csv')
manager_survey_data = load_data('manager_survey_data.csv')
in_time = load_data('in_time.csv')
out_time = load_data('out_time.csv')
general_data = load_data('general_data.csv')

# Choix des données

## Éthique

Dans le cas de notre modèle et de l'utilisation de nos données il est important de déterminer étiquement la conservation de certaines données ou non. 

Pour cela nous avons procédé à un brainstorming et à une lecture des recommendations de la CNIL pour conserver uniquement les données nécessaires et ne prétant pas à une possible discrimination.

Voici la liste des données que nous ne souhaitons pas conserver pour notre modèle :

- **L’âge des employés (Age) :** Nous souhaitons rester dans la plus grande neutralité possible.Effectivement, l’âge ne doit pas nous permettre de définir si une personne est plus à même de quitter l’entreprise ou non. 
<br>

- **Le genre des employés (Gender) :** Le genre est une donnée non pertinante sur les critères qui pousserai à un turnover. Cette donnée pourrait être discrimante 
<br>

- **Le statut marital (MaritalStatus) :** Cette donnée ne nous permetrai pas d’interpreter des critères cohérent concernant les Turn-over dans l’entrprise. Ce serai une surinterpretation des données fournis.

In [ ]:
#remove the columns for ethic
general_dataset = general_data.copy() #copy the dataframe to avoid changing the original one
general_dataset.drop("Age", axis=1, inplace=True) 
general_dataset.drop("Gender", axis=1, inplace=True)
general_dataset.drop("MaritalStatus", axis=1, inplace=True)

## Logique

Il est également nécessaire de déterminer d'une manière logique les données utile à conserver ou non. 

Pour cela il suffit d'isoler dans un premier temps les colonnes de la table ***General_data***  ou le champs de valeur est égal à 1, c'est à dire que chaque données est la même.

Pour cela il suffit d'executer cette fonction afin de déterminer les champs répondant à cette condition :

In [ ]:
for col in general_data.columns:
    value = general_data[col].nunique()
    value_of = general_data[col].unique()
    if value == 1:
        print(col)
        print(value_of)

On constate alors que :

- **EmployeeCount :** Cette colonne correspond à la présence ou non d'un employé dans l'entreprise dans les effectifs en 2015, on constate que chaque ligne du tableau est à 1 cela signifie donc que tous les employés présent dans les données était dans l'entreprise en 2015, il n'est donc pas intéressant de conserver cette donnée.
<br>

- **Over18 :** Cette colonne indique si l'age de l'employé est supérieure ou non à 18 ans, on constate que chaque ligne du tableau est à la valeure "Y" cela signifie donc que en 2015 chaque employé était majeur, il n'est donc pas intéressant de conserver cette donnée. 
<br>

- **StandardHours :** Cette colonne spécifie le nombre d'heure inscrite sur le contrat de l'employé, chaque ligne étant positionnée à 8h on en déduit que tous les employés disposait du même type de contrat en 2015. Il n'est donc pas intéressant de conserver cette donnée.

<div class="alert alert-block alert-info">
<b>Info:</b> Cette étude à également été effectuée sur les autres tables mais aucune valeur unique est apparue, nous conserverons alors toutes les données de celles-ci.
</div>


In [ ]:
# remove the non revelent columns
general_dataset.drop("Over18", axis=1, inplace=True)
general_dataset.drop("EmployeeCount", axis=1, inplace=True)
general_dataset.drop("StandardHours", axis=1, inplace=True)

## Conclusion

Après avoir selectionné les données à conserver en fonction de différents facteurs voici une liste exhaustive de celles conservées pour la réalisation de notre modèle :

###  <font color='green'> General_data </font>

- **Attrition :** L'objet de notre étude, est-ce que l'employé a quitté l'entreprise durant l'année 2016 ?
<br>

- **BusinessTravel :** A quel fréquence l'employé a été amené à se déplacer dans le cadre de son travail en 2015 ? (Non-Travel = jamais, Travel_Rarely= rarement, Travel_Frequently = fréquemment)
<br>

- **DistanceFromHome :** Distance en km entre le logement de l'employé et l'entreprise.
<br>

- **Education : Niveau d'étude :** 1=Avant College (équivalent niveau Bac), 2=College (équivalent Bac+2), 3=Bachelor (Bac+3), 4=Master (Bac+5) et 5=PhD (Thèse de doctorat).
<br>

- **EducationField :** Domaine d'étude, matière principale
<br>

- **EmployeeId :** l'identifiant d'un employé
<br>

- **JobLevel :** Niveau hiérarchique dans l'entreprise de 1 à 5
<br>

- **JobRole :** Métier dans l'entreprise
<br>

- **MonthlyIncome :** Salaire brut en roupies par mois
<br>

- **NumCompaniesWorked :** Nombre d'entreprises pour lequel le salarié a travaillé avant de rejoindre HumanForYou.
<br>

- **PercentSalaryHike :** % d'augmentation du salaire en 2015.
<br>

- **StockOptionLevel :** Niveau d'investissement en actions de l'entreprise par le salarié.
<br>

- **TotalWorkingYears :** Nombre d'années d'expérience en entreprise du salarié pour le même type de poste.
<br>

- **TrainingTimesLastYear :** Nombre de jours de formation en 2015
<br>

- **YearsAtCompany :** Ancienneté dans l'entreprise
<br>

- **YearsSinceLastPromotion :** Nombre d'années depuis la dernière augmentation individuelle
<br>

- **YearsWithCurrentManager :** Nombre d'années de collaboration sous la responsabilité du manager actuel de l'employé.

###  <font color='green'> Employee_survey_data </font>

- **L'environnement de travail :** : noté 1 ("Faible"), 2 ("Moyen"), 3 ("Élevé") ou 4 ("Très élevé") : EnvironmentSatisfaction
<br>

- **Son travail :** noté de 1 à 4 comme précédemment : JobSatisfaction
<br>

- **Son équilibre entre vie professionnelle et vie privée :** noté 1 ("Mauvais"), 2 ("Satisfaisant"), 3 ("Très satisfaisant") ou 4 ("Excellent") : WorkLifeBalance

###  <font color='green'> Manager_survey_data </font>

- **Une évaluation de son implication dans son travail :** notée 1 ('Faible'), 2 ("Moyenne"), 3 ("Importante") ou 4 ("Très importante") : JobInvolvement
<br>

- **Une évaluation de son niveau de performance annuel pour l'entreprise :** notée 1 ("Faible"), 2 ("Bon"), 3 ("Excellent") ou 4 ("Au delà des attentes") : PerformanceRating

## Encodage

### Encodage binaire de l'attribut Attrition
On convertit les données de l'attribut Attrition, les 'Yes' et 'No' deviennent respectivement des 1 et des 0.

In [ ]:
general_dataset['Attrition'] = general_dataset['Attrition'].map({'Yes': 1, 'No':0}) # convertir la colonne Attrition en 0 et 1

fir_column = general_dataset.pop('Attrition') # retirer la colonne Attrition
general_dataset.insert(0 ,'Attrition', fir_column) # la remettre en première colonne

### One-Hot Encoding
Nous allons créer des variables supplémentaires pour représenter chacun des catégories.
**BusinessTravel**, **Department**, **EducationField**, **JobRole**

In [ ]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False) # sparse=False pour obtenir un tableau numpy (et non une matrice creuse)
data_encoded = onehot_encoder.fit_transform(general_dataset[['BusinessTravel', 'Department', 'EducationField', 'JobRole']])
onehot_encoder.categories_ # affiche les catégories

## Echantillonage stratifié

Nous désirons effectuer un échantillonage respectant les proportions de représentation des différentes catégories de salaire. bla bla bla

In [ ]:
general_dataset["MonthlyIncome"].describe()

max à 200 000 on veux 10 cat donc /20 000

In [ ]:
general_dataset["MonthlyIncome_cat"] = np.ceil(general_dataset["MonthlyIncome"]/20000)

general_dataset["MonthlyIncome_cat"].hist(facecolor = '#ff0005', edgecolor='#000000', linewidth=0.8, bins=10)

suppr la col d'origine

In [ ]:
general_dataset.drop("MonthlyIncome", axis=1, inplace=True)
general_dataset.head(3)

# Calcul temps de travail moyen + temps arrivé et temps départ moyen

De nombreuses données sont manquantes parmis les horaires. Nous faisons le choix de supprimer une colonne contenant plus de 50% de données manquantes. Pour les autres données manquantes, nous les remplaçons par la date Unix qui correspond à une valeur nul en format date afin que ces données n'aient pas d'influence sur nos calculs.

In [ ]:
#prepare time dataset
def dropNan(dataset):

    data = dataset.copy()
    colNames = [col for col in data.columns]

    percent= [round(100-((data[col].count()/len(data.index))*100),2) for col in data.columns]
    indexes = [index for index, value in enumerate(percent) if value > 50]

    for i in indexes:
        data.drop(colNames[i], axis=1, inplace=True)
        
    data.fillna("1970-01-01 00:00:00", inplace=True)
    
    return data

In [ ]:
inTime_prep = dropNan(in_time)
outTime_prep = dropNan(out_time)
inTime_prep.head(3)

In [ ]:
def sec(date):
    getlast = ((str(date))[-8:]).replace(":", "")
    time = (int(getlast[4:6])) + ((int(getlast[2:4])) * 60) + (int(getlast[0:2])) * 3600
    return time

def average_In_Out_Time(time):
    tabs = []
    for row in range(0, len(time)):
        means = [sec(dt.datetime.strptime(time.iloc[row, col], "%Y-%m-%d %H:%M:%S")) for col in range(1, len(time.columns))]
        means_convert = round(((int((round(mean(means), 0)))) / 3600),2)
        tabs += [means_convert]
    return tabs
    
def average_worktime(out_time, in_time) :
    tab = []
    for lines in range(0, len(out_time)):
        worktime = [(((dt.datetime.strptime(out_time.iloc[lines,col], "%Y-%m-%d %H:%M:%S")) - (dt.datetime.strptime(in_time.iloc[lines,col], "%Y-%m-%d %H:%M:%S"))).total_seconds()) for col in range(1, len(out_time.columns))]
        worktime_convert = round(((int((round(mean(worktime), 0)))) / 3600),2)
        tab += [worktime_convert]
    return tab

In [ ]:
d = {'EmployeeID': inTime_prep.iloc[:, 0], 'Average_Worktime': average_worktime(outTime_prep, inTime_prep), 'Average_In_Time': average_In_Out_Time(inTime_prep), 'Average_Out_Time': average_In_Out_Time(outTime_prep)}
wortime_employee = pd.DataFrame(data=d)
wortime_employee.head(3)

# Fusion des tables
Nous allons fusionner les dataframes selon l'id de l'employé pour inclure les evaluations d'impliquation, les évaluations de niveaux de performance et les données concernant les horaires.

In [ ]:
#merge the dataframes
survey_dataset = pd.merge(employee_survey_data, manager_survey_data, on='EmployeeID')
dataset_with_survey = pd.merge(general_dataset, survey_dataset, on='EmployeeID')
Dataset = pd.merge(dataset_with_survey, wortime_employee, on='EmployeeID')
Dataset.head(3)

# Nettoyage des données

Nous allons supprimer les valeurs manquantes.
Nous constatons qu'il y a des valeurs manquantes pour 110 employés.

In [ ]:
sample_incomplete_rows = Dataset[Dataset.isnull().any(axis=1)]
len(sample_incomplete_rows.index)

Cela correspond à moins de 3% de nos valeurs.

Nous faisons donc le choix de supprimer les lignes contenant des valeurs manquantes.

In [ ]:
(len(sample_incomplete_rows.index)/len(Dataset.index))*100

In [ ]:
Dataset.dropna(inplace=True)
Dataset.head(3)

# Correlation

In [ ]:
corrmatrix = Dataset.corr()
corrmatrix['Attrition']

In [ ]:
attributes = ["Attrition", "Average_Worktime", "Average_Out_Time", "YearsWithCurrManager", "YearsAtCompany", "TotalWorkingYears"]
scat_matrix = scatter_matrix(Dataset[attributes], figsize=(15, 12))

In [ ]:
Dataset.plot(kind="scatter", x="Attrition", y="Average_Worktime", alpha=0.1)

## Pipeline de transformation test


In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator

class EthicColumnsRemover(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy.drop("Age", axis=1, inplace=True) 
        X_copy.drop("Gender", axis=1, inplace=True)
        X_copy.drop("MaritalStatus", axis=1, inplace=True)
        X_copy.drop("Over18", axis=1, inplace=True)
        X_copy.drop("EmployeeCount", axis=1, inplace=True)
        X_copy.drop("StandardHours", axis=1, inplace=True)
        return X_copy.values

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

dataset_pipeline = Pipeline([
    ('removeEthicColumns', EthicColumnsRemover()),
    ('onehot_encoder', OneHotEncoder(sparse=False)),
    ('scaler', StandardScaler())
])

transformed_data = dataset_pipeline.fit_transform(general_data)
transformed_data
